Link to step 6 [Notebook_6](https://github.com/fatsoengineer/Computer_Vision_Course/blob/master/S5/6th_Model_Layer_image_aug.ipynb)
## Step 7

 **Target:**

> Adding LR scheduler


**Results**


> 
*   Total params: 6,376
*   Best Test Accuracy: 
```
EPOCH:14 | LR: [0.008679444396382234]
Loss=0.018181532621383667 Batch_id=937 Accuracy=99.04: 100%|██████████| 938/938 [00:16<00:00, 55.82it/s]
  0%|          | 0/938 [00:00<?, ?it/s]
Test set: Average loss: 0.0178, Accuracy: 9947/10000 (99.47%)
```


**Analysis**

> * Tried Step LR or ReduceLrOnPlateau schedulers to check the results, but got the best result with OneCycleLR.
* Ran multiple times to confirm that the results are not fluke or not fluctuating from each training at large 


In [0]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [0]:
cd /content/drive/My Drive/Backup/EVP/Session_6/L2_misclassified

/content/drive/My Drive/Backup/EVP/Session_6/L2_misclassified


# Import Libraries

In [0]:
from __future__ import print_function
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms

!pip install torchsummary
from torchsummary import summary
import torchvision.transforms.functional as TF
!!pip install mlcrate
import mlcrate as mlc

## Data Transformations

We first start with defining our data transformations. We need to think what our data is and how can we augment it to correct represent images which it might not see otherwise. 


In [0]:
# Train Phase transformations
train_transforms = transforms.Compose([
                                      #  transforms.Resize((28, 28)),
                                      #  transforms.ColorJitter(brightness=0.10, contrast=0.1, saturation=0.10, hue=0.1),
                                       transforms.RandomRotation((-5.0, 5.0), fill=(1,)),
                                       transforms.ToTensor(),
                                       transforms.Normalize((0.1307,), (0.3081,)), # The mean and std have to be sequences (e.g., tuples), therefore you should add a comma after the values. 
                                       # Note the difference between (0.1307) and (0.1307,)
                                      #  transforms.RandomErasing()
                                       ])

# Test Phase transformations
test_transforms = transforms.Compose([
                                      #  transforms.Resize((28, 28)),
                                      #  transforms.ColorJitter(brightness=0.10, contrast=0.1, saturation=0.10, hue=0.1),
                                       transforms.ToTensor(),
                                       transforms.Normalize((0.1307,), (0.3081,))
                                       ])


# Dataset and Creating Train/Test Split

In [0]:
train_data = datasets.MNIST('./data', train=True, download=True, transform=train_transforms)
test_data = datasets.MNIST('./data', train=False, download=True, transform=test_transforms)


Extracting ./data/MNIST/raw/train-images-idx3-ubyte.gz to ./data/MNIST/raw



Extracting ./data/MNIST/raw/train-labels-idx1-ubyte.gz to ./data/MNIST/raw



Extracting ./data/MNIST/raw/t10k-images-idx3-ubyte.gz to ./data/MNIST/raw



Extracting ./data/MNIST/raw/t10k-labels-idx1-ubyte.gz to ./data/MNIST/raw
Processing...
Done!


# Dataloader Arguments & Test/Train Dataloaders


In [0]:
SEED = 1

# CUDA?
cuda = torch.cuda.is_available()
print("CUDA Available?", cuda)

# For reproducibility
torch.manual_seed(SEED)

if cuda:
    torch.cuda.manual_seed(SEED)

# dataloader arguments - something you'll fetch these from cmdprmt
dataloader_args = dict(shuffle=True, batch_size=64, num_workers=4, pin_memory=True) if cuda else dict(shuffle=True, batch_size=64)

# train dataloader
train_loader = torch.utils.data.DataLoader(train_data, **dataloader_args)

# test dataloader
test_loader = torch.utils.data.DataLoader(test_data, **dataloader_args)

CUDA Available? True


# Data Statistics

It is important to know your data very well. Let's check some of the statistics around our data and how it actually looks like

In [0]:
# # We'd need to convert it into Numpy! Remember above we have converted it into tensors already
# train_data = train.train_data
# train_data = train.transform(train_data.numpy())

# print('[Train]')
# print(' - Numpy Shape:', train.train_data.cpu().numpy().shape)
# print(' - Tensor Shape:', train.train_data.size())
# print(' - min:', torch.min(train_data))
# print(' - max:', torch.max(train_data))
# print(' - mean:', torch.mean(train_data))
# print(' - std:', torch.std(train_data))
# print(' - var:', torch.var(train_data))

# dataiter = iter(train_loader)
# images, labels = dataiter.next()

# print(images.shape)
# print(labels.shape)

# Let's visualize some of the images
%matplotlib inline
import matplotlib.pyplot as plt

# plt.imshow(images[0].numpy().squeeze(), cmap='gray_r')


## MORE

It is important that we view as many images as possible. This is required to get some idea on image augmentation later on

In [0]:
# figure = plt.figure()
# num_of_images = 60
# for index in range(1, num_of_images + 1):
#     plt.subplot(6, 10, index)
#     plt.axis('off')
#     plt.imshow(images[index].numpy().squeeze(), cmap='gray_r')


# The model
Let's start with the model we first saw

In [0]:
dropout_value =.10

class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()

        self.conv1 = nn.Sequential(
            nn.Conv2d(in_channels=1, out_channels=8, kernel_size=(3, 3), padding=0, bias=False), #26
            nn.BatchNorm2d(8),
            nn.ReLU(),
            nn.Dropout(dropout_value),

            nn.Conv2d(in_channels=8, out_channels=12, kernel_size=(3, 3), padding=0, bias=False), #24
            nn.BatchNorm2d(12),
            nn.ReLU(),
            nn.Dropout(dropout_value),

            # nn.Conv2d(in_channels=12, out_channels=12, kernel_size=(3, 3), padding=0, bias=False), #22
            # nn.BatchNorm2d(12),
            # nn.ReLU(),
            # nn.Dropout(dropout_value),

            nn.MaxPool2d(2,2), # 11
        )

        self.conv2 = nn.Sequential(
            nn.Conv2d(in_channels=12, out_channels=12, kernel_size=(3, 3), padding=0, bias=False), #9
            nn.BatchNorm2d(12),
            nn.ReLU(),
            nn.Dropout(dropout_value),

            nn.Conv2d(in_channels=12, out_channels=12, kernel_size=(3, 3), padding=0, bias=False), #7
            nn.BatchNorm2d(12),
            nn.ReLU(),
            nn.Dropout(dropout_value)
        )

        self.conv3 = nn.Sequential(
            nn.Conv2d(in_channels=12, out_channels=12, kernel_size=(3, 3), padding=0, bias=False),
            nn.BatchNorm2d(12),
            nn.ReLU(),
            nn.Dropout(dropout_value),

            nn.Conv2d(in_channels=12, out_channels=12, kernel_size=(3, 3), padding=0, bias=False),
            nn.BatchNorm2d(12),
            nn.ReLU(),
            nn.Dropout(dropout_value),
        )

        self.gap = nn.Sequential(
            nn.AvgPool2d(kernel_size=4)
        ) # output_size = 1

        self.reducer = nn.Conv2d(12,10,1,bias=False)

    def forward(self, x):
      x= self.conv1(x)
      x= self.conv2(x)
      x= self.conv3(x)
      x =self.gap(x)
      x= self.reducer(x)
      x = x.view(-1, 10)
      return F.log_softmax(x)

Batch Norm after Relu


# Model Params
Can't emphasize on how important viewing Model Summary is. 
Unfortunately, there is no in-built model visualizer, so we have to take external help

In [0]:
use_cuda = torch.cuda.is_available()
device = torch.device("cuda" if use_cuda else "cpu")
print(device)
model = Net().to(device)
summary(model, input_size=(1, 28, 28))

cuda
----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1            [-1, 8, 26, 26]              72
       BatchNorm2d-2            [-1, 8, 26, 26]              16
              ReLU-3            [-1, 8, 26, 26]               0
           Dropout-4            [-1, 8, 26, 26]               0
            Conv2d-5           [-1, 12, 24, 24]             864
       BatchNorm2d-6           [-1, 12, 24, 24]              24
              ReLU-7           [-1, 12, 24, 24]               0
           Dropout-8           [-1, 12, 24, 24]               0
         MaxPool2d-9           [-1, 12, 12, 12]               0
           Conv2d-10           [-1, 12, 10, 10]           1,296
      BatchNorm2d-11           [-1, 12, 10, 10]              24
             ReLU-12           [-1, 12, 10, 10]               0
          Dropout-13           [-1, 12, 10, 10]               0
           Conv2d-14             [

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:63: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.


# Training and Testing

Looking at logs can be boring, so we'll introduce **tqdm** progressbar to get cooler logs. 

Let's write train and test functions

In [0]:

from tqdm import tqdm




def train(model, device, train_loader, optimizer, epoch,scheduler, factor):
  # train_losses = []
  # train_acc = []

  model.train()
  pbar = tqdm(train_loader)
  correct = 0
  processed = 0
  train_loss = 0
  train_acc = 0
  for batch_idx, (data, target) in enumerate(pbar):
    # get samples
    data, target = data.to(device), target.to(device)

    # Init
    optimizer.zero_grad()
    # In PyTorch, we need to set the gradients to zero before starting to do backpropragation because PyTorch accumulates the gradients on subsequent backward passes. 
    # Because of this, when you start your training loop, ideally you should zero out the gradients so that you do the parameter update correctly.

    # Predict
    y_pred = model(data)

    # Calculate loss
    loss = F.nll_loss(y_pred, target)
    # train_losses.append(loss)
    train_loss += F.nll_loss(y_pred, target, reduction='sum').item()  # sum up batch loss
    
    # Backpropagation
    loss.backward()
    optimizer.step()

    # Update pbar-tqdm
    
    pred = y_pred.argmax(dim=1, keepdim=True)  # get the index of the max log-probability
    correct += pred.eq(target.view_as(pred)).sum().item()
    processed += len(data)

    pbar.set_description(desc= f'Loss={loss.item()} Batch_id={batch_idx} Accuracy={100*correct/processed:0.2f}')
    
    # train_acc.append(100*correct/processed)
    scheduler.step()
  train_acc = 100*correct/processed
  train_loss /= len(train_loader.dataset)
  return train_loss, train_acc
    
def test(model, device, test_loader):
  # test_losses = []

  # test_acc = []
  misclassified_images_list = []
  model.eval()
  test_loss = 0
  test_acc = 0
  correct = 0
  with torch.no_grad():
      for data, target in test_loader:
          data, target = data.to(device), target.to(device)
          # print(data.shape)
          output = model(data)
          test_loss += F.nll_loss(output, target, reduction='sum').item()  # sum up batch loss
          pred = output.argmax(dim=1, keepdim=True)  # get the index of the max log-probability
          correct += pred.eq(target.view_as(pred)).sum().item()

          # Check images
          wrong_idx = (pred != target.view_as(pred)).nonzero()[:, 0]
          wrong_samples = data[wrong_idx]
          pred_label_list = pred[wrong_idx]
          actual_label_list = target.view_as(pred)[wrong_idx]

          for idx, img in enumerate(wrong_samples):
              img = img.cpu()
              pred_label = pred_label_list[idx].cpu()
              actual_label = actual_label_list[idx].cpu()

              # Undo normalization
              img = img * 0.3081
              img = img + 0.1307
              img = img * 255.
              img = img.byte()
              img = TF.to_pil_image(img)
              misclassified_images_list.append({
                  "image": img,
                  "actual_label": actual_label,
                  "predicted_label": pred_label,
              })


  test_loss /= len(test_loader.dataset)
  # test_losses.append(test_loss)
  test_acc = 100. * correct / len(test_loader.dataset)

  print('\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.2f}%)\n'.format(
      test_loss, correct, len(test_loader.dataset),
      test_acc))
  # test_acc.append(acc)

  return test_loss, test_acc, misclassified_images_list


In [0]:
def run_model(model, device, train_loader, optimizer, scheduler, test_loader, EPOCHS, logger, vals, factor):
  train_loss_list = []
  train_acc_list = []
  test_losses_list = []
  test_acc_list = []
  for epoch in range(1,EPOCHS+1):
      print(f"EPOCH:{epoch} | LR: {scheduler.get_lr()}")
      train_loss, train_acc= train(model, device, train_loader, optimizer, epoch, scheduler, factor)
      test_loss, test_acc, misclassified_images_list= test(model, device, test_loader)
      train_loss_list.append(train_loss)
      train_acc_list.append(train_acc)
      test_losses_list.append(test_loss)
      test_acc_list.append(test_acc)
      logger.write([vals[0], vals[1], epoch, optimizer.param_groups[0]['lr'], train_loss, test_loss, train_acc, test_acc])
  return train_loss_list, train_acc_list, test_losses_list, test_acc_list, misclassified_images_list

# Let's Train and test our model

In [0]:
def plot_loss_graph(train_losses, test_losses, weight_decay):
  plt.figure(figsize=[10,6])
  plt.plot(train_losses)
  plt.plot(test_losses)
  plt.legend(['Train Loss', 'Test Loss'])
  plt.xlabel('Epoch')
  plt.ylabel('Loss')
  plt.title(f"L2 with {weight_decay} weight decay Model Loss")
  plt.savefig(f"L2 with {weight_decay} weight decay Model Loss.png")

In [0]:
def plot_acc_graph(train_acc, test_acc, weight_decay):
  plt.figure(figsize=[10,6])
  plt.plot(train_acc)
  plt.plot(test_acc)
  plt.legend(['Train Acc', 'Test Acc'])
  plt.xlabel('Epoch')
  plt.ylabel('Acc')
  plt.title(f"L2 with {weight_decay} weight decay Model Accuracy")
  plt.savefig(f"L2 with {weight_decay} weight decay Model Accuracy.png")

In [0]:
def create_misclassified_image(weight_decay, misclassified_images_list):
  rows =5
  cols =5
  total_images =rows*cols

  fig = plt.figure(figsize=(10, 15))

  for i in range(total_images):
    fig.add_subplot(rows, cols, i+1)
    plt.imshow(misclassified_images_list[i]["image"])
    plt.xlabel('Pred {} Actual {}'.format(misclassified_images_list[i]["actual_label"].item(),
                                          misclassified_images_list[i]["predicted_label"].item()))

  fig.savefig(f"L2 with {weight_decay} weight decay misclassified images.png")

In [0]:
def plot_weight_decay_comparator_accuracy(train_acc_list, test_acc_list, weight_decay_list):
  # Training Data
  lgd_train = [f"Train Accuracy for weight decay {_}" for _ in weight_decay_list]
  plt.figure(figsize=[10,6])
  for lst in train_acc_list:
    plt.plot(lst)

  plt.legend(lgd_train)
  plt.xlabel('Epoch')
  plt.ylabel('Acc')
  plt.title(f"L2 weight decay Model Accuracy For Training Data")
  plt.savefig(f"L2 weight decay Model Accuracy For Training Data.png")

  # Test data
  lgd_train = [f"Test Accuracy for weight decay {_}" for _ in weight_decay_list]
  plt.figure(figsize=[10,6])
  for lst in test_acc_list:
    plt.plot(lst)

  plt.legend(lgd_train)
  plt.xlabel('Epoch')
  plt.ylabel('Acc')
  plt.title(f"L2 weight decay Model Accuracy For Test Data")
  plt.savefig(f"L2 weight decay Model Accuracy For Test Data.png")

In [0]:
def plot_weight_decay_comparator_loss(train_losses_list, test_losses_list, weight_decay_list):
  # Training Data
  lgd_train = [f"Train loss for weight decay {_}" for _ in weight_decay_list]
  plt.figure(figsize=[10,6])
  for lst in train_losses_list:
    plt.plot(lst)

  plt.legend(lgd_train)
  plt.xlabel('Epoch')
  plt.ylabel('Loss')
  plt.title(f"L2 weight decay Model loss For Training Data")
  plt.savefig(f"L2 weight decay Model loss For Training Data.png")

  # Test data
  lgd_train = [f"Test loss for weight decay {_}" for _ in weight_decay_list]
  plt.figure(figsize=[10,6])
  for lst in test_losses_list:
    plt.plot(lst)

  plt.legend(lgd_train)
  plt.xlabel('Epoch')
  plt.ylabel('Loss')
  plt.title(f"L2 weight decay Model loss For Test Data")
  plt.savefig(f"L2 weight decay Model loss For Test Data.png")

In [0]:
def train_model(weight_decay, EPOCHS):

  model =  Net().to(device)
  # weight_decay = 0.001
  optimizer = optim.SGD(model.parameters(), lr=0.01, momentum=0.9, weight_decay=weight_decay )
  scheduler = torch.optim.lr_scheduler.OneCycleLR(optimizer, max_lr=0.1, steps_per_epoch=len(train_loader), epochs=EPOCHS)

  timer = mlc.time.Timer()
  header = ["iteration_type",'weight_decay' ,'epoch', 'lr', 'train_loss', 'val_loss', 'train_acc', 'val_acc']
  vals = ["l2", weight_decay]
  logger = mlc.LinewiseCSVWriter(f'train_log_l2_{weight_decay}.csv', header=header)

  train_losses, train_acc, test_losses, test_acc, misclassified_images_list = run_model(model, device, train_loader, optimizer, scheduler, test_loader, EPOCHS, logger, vals, weight_decay)
  plot_loss_graph(train_losses, test_losses, weight_decay)
  plot_acc_graph(train_acc, test_acc, weight_decay)
  create_misclassified_image(weight_decay, misclassified_images_list)
  return train_losses, train_acc, test_losses, test_acc

In [21]:
EPOCHS = 40
weight_decay_list = [
                     0.1, 0.01, 0.001, 0.0005, 0.0001
]
train_losses_list, train_acc_list, test_losses_list, test_acc_list=[],[],[],[]#train_losses, train_acc, test_losses, test_acc
for weight_decay in weight_decay_list:
  print(f"Starting model train for weight decay {weight_decay}")
  train_losses, train_acc, test_losses, test_acc = train_model(weight_decay, EPOCHS)
  train_losses_list.append(train_losses)
  train_acc_list.append(train_acc)
  test_losses_list.append(test_losses)
  test_acc_list.append(test_acc)
plot_weight_decay_comparator_accuracy(train_acc_list, test_acc_list, weight_decay_list)
plot_weight_decay_comparator_loss(train_losses_list, test_losses_list, weight_decay_list)

  0%|          | 0/938 [00:00<?, ?it/s]

Starting model train for weight decay 0.1
EPOCH:1 | LR: [0.0040000000000000036]


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:63: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.
Loss=1.4493075609207153 Batch_id=937 Accuracy=55.00: 100%|██████████| 938/938 [00:25<00:00, 37.33it/s]
  0%|          | 0/938 [00:00<?, ?it/s]


Test set: Average loss: 1.3517, Accuracy: 6476/10000 (64.76%)

EPOCH:2 | LR: [0.005635849325882514]


Loss=1.2332388162612915 Batch_id=937 Accuracy=73.25: 100%|██████████| 938/938 [00:25<00:00, 37.19it/s]
  0%|          | 0/938 [00:00<?, ?it/s]

...


Test set: Average loss: 2.0781, Accuracy: 4374/10000 (43.74%)

EPOCH:19 | LR: [0.0890878885269022]


Loss=0.19484251737594604 Batch_id=937 Accuracy=93.83: 100%|██████████| 938/938 [00:26<00:00, 35.66it/s]
  0%|          | 0/938 [00:00<?, ?it/s]


Test set: Average loss: 0.2479, Accuracy: 9390/10000 (93.90%)

EPOCH:20 | LR: [0.08535116834038248]


Loss=0.13652047514915466 Batch_id=606 Accuracy=93.87:  64%|██████▍   | 602/938 [00:17<00:09, 36.07it/s]

Buffered data was truncated after reaching the output size limit.